In [407]:
# En primer lugar se instalan e importan las librerías y dependencias que serán utilizadas en el proceso.
!pip install SPARQLWrapper  
!pip install wikipedia      
import pandas as pd
from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import wikipedia
import json
import xml.etree.ElementTree as ET

In [416]:
# A continuación se pregunta al usuario por el identificador de la clase del cual se quiere generar un diagrama
print("¿Cual es el identificador de la clase del cual se quiere generar un diagrama?")
clase = input()

¿Cual es el identificador de la clase del cual se quiere generar un diagrama?
Q349


In [417]:
#Se define la función para consultar Wikidata. Se consulta el endpoint señalado y 
#devuelve los resultados como un DataFrame de Pandas.

def query_wikidata(sparql_query, sparql_service_url):        
  
    # crear la conexión con el endpoint
    sparql = SPARQLWrapper(sparql_service_url)    
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # solicitar resultados
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

#Se genera la consulta para obtener el numero de instancias de la clase a representar
sparql_query1 = """SELECT DISTINCT ?clase ?claseLabel
WHERE {
?clase wdt:P31 wd:"""+clase+""".
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}
"""
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
instancias_wikidata = query_wikidata(sparql_query1, sparql_service_url)

In [418]:
#Se confirma que la consulta haya salido bien
instancias_wikidata.head()

,clase.type,clase.value,claseLabel.type,claseLabel.value,claseLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q581356,literal,Q581356,NaN
1,uri,http://www.wikidata.org/entity/Q627949,literal,Q627949,NaN
2,uri,http://www.wikidata.org/entity/Q638269,literal,Q638269,NaN
3,uri,http://www.wikidata.org/entity/Q895138,literal,Q895138,NaN
4,uri,http://www.wikidata.org/entity/Q907328,literal,Q907328,NaN


In [419]:
#Se obtiene el numero de instancias de la clase de entrada
numero_de_instancias = instancias_wikidata.shape[0]
numero_de_instancias

201

In [420]:
#Se genera la consulta para obtener las propiedades de la clase de entrada
sparql_query2 = """SELECT DISTINCT ?p ?pLabel
WHERE {
?clase wdt:P31 wd:"""+clase+""";
?p ?x.
#to obtain property labels:
?property wikibase:directClaim ?p
#to obtain labels in spanish:
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}   
    """
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
propiedades_wikidata = query_wikidata(sparql_query2, sparql_service_url)

In [421]:
#Se confirma que la consulta haya salido bien
propiedades_wikidata.head()

,p.type,p.value,pLabel.type,pLabel.value
0,uri,http://www.wikidata.org/prop/direct/P279,literal,http://www.wikidata.org/prop/direct/P279
1,uri,http://www.wikidata.org/prop/direct/P276,literal,http://www.wikidata.org/prop/direct/P276
2,uri,http://www.wikidata.org/prop/direct/P349,literal,http://www.wikidata.org/prop/direct/P349
3,uri,http://www.wikidata.org/prop/direct/P373,literal,http://www.wikidata.org/prop/direct/P373
4,uri,http://www.wikidata.org/prop/direct/P495,literal,http://www.wikidata.org/prop/direct/P495


In [422]:
#Se genera la consulta para obtener las propiedades de la clase.
sparql_query2 = """SELECT DISTINCT ?p ?pLabel
WHERE {
?clase wdt:P31 wd:"""+clase+""";
?p ?x.
#to obtain property labels:
?property wikibase:directClaim ?p
#to obtain labels in spanish:
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}   
    """
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
propiedades_wikidata = query_wikidata(sparql_query2, sparql_service_url)

#Se listan las propiedades de salida de la clase y se le asignan a la variable "lista_propiedades"
lista_completa = propiedades_wikidata["p.value"].tolist()
lista_propiedades = []
for ele in lista_completa:
    ele = ele[36:]
    lista_propiedades.append(ele)

In [424]:
lista_resultado_propiedades_clases = []
#Para cada propiedad de la lista:
for prop in lista_propiedades:
    
    #Se obtienen de las propiedades anteriores aquellas que son object properties. Se almacenan en una lista.
    #Aquí se genera la consulta. Esta se hace para ver si la propiedad con la que se esta tratando es object property o no.
    sparql= SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery("""ASK WHERE {
    wdt:"""+prop+""" rdf:type owl:ObjectProperty
    }   
    """)
    sparql.setReturnFormat(XML)
    results = sparql.query().convert()
    string = results.toxml()
    
    #Se obtiene el valor booleano del xml string, siendo este "true" en caso afirmativo
    es_OP = string[105:109]
    
    #En caso de que la variable es_OP sea "true", es decir la propiedad sea object propery:
    if es_OP == "true":
        
        #Se genera la consulta. Se realiza esta consulta para obtener las clases que pertenecen a dicha propiedad.
        #Se realiza esta consulta para todas las object properties encontradas anteriormente.
        sparql1 = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql1.setQuery("""
        SELECT DISTINCT ?claseInicial ?claseInicialLabel ?prop ?propLabel ?Clase ?ClaseLabel
        WHERE {
        #occupation (P106) must be Formula 1 driver (Q10841764)
        ?claseInicial wdt:P31 wd:"""+clase+""" .
        ?claseInicial wdt:"""+prop +""" ?prop .
        ?prop wdt:P31 ?Clase .
        #to obtain labels in spanish:
        SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
        }  
        """)
        sparql1.setReturnFormat(JSON)
        results = sparql1.query().convert()
        
        #Se añade a la lista "lista_clases_para_una_propiedad_completo" todas las clases que resultan de la consulta anterior. 
        lista_clases_para_una_propiedad_completo = []
        for result in results["results"]["bindings"]:
            lista_clases_para_una_propiedad_completo.append(result["Clase"]["value"])
        
        #Al tener estos valores en su inicio la URL "https:..", esto se elimina dejando solo el identificador de la clase.
        lista_clases_para_una_propiedad = []
        for ele in lista_clases_para_una_propiedad_completo:
            ele = ele[31:]
            lista_clases_para_una_propiedad.append(ele)
        
        #Se tiene la lista "lista_clases_para_una_propiedad" en la cual se encuentran todas las clases de la propiedad 
        #estudiada.
        
        #Se obtienen las value type constraint de las propiedades. La consulta siguiente las obtiene.
        sparql2 = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql2.setQuery("""SELECT ?Property_ ?Property_Label ?Property_Description ?class_ ?class_Label ?relation_ ?relation_Label
        WHERE {
        ?Property_ wikibase:directClaim wdt:"""+prop+""" .
        ?Property_ p:P2302 ?constraint_statement .
        ?constraint_statement ps:P2302 wd:Q21510865 .
        OPTIONAL {?constraint_statement pq:P2308 ?class_ .}
        OPTIONAL {?constraint_statement pq:P2309 ?relation_ .}

        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        } 
        """)
        sparql2.setReturnFormat(JSON)
        results = sparql2.query().convert()

        #En la lista "lista_value_type_constraints_completa" se almacenan todas las value type constraint de la propiedad 
        #estudiada.
        lista_value_type_constraints_completa = []
        for result in results["results"]["bindings"]:
            lista_value_type_constraints_completa.append(result["class_"]["value"])
            
        #Al tener estos valores en su inicio la URL "https:..", esto se elimina dejando solo el identificador de la clase.
        lista_value_type_constraints = []
        for ele in lista_value_type_constraints_completa:
            ele = ele[31:]
            lista_value_type_constraints.append(ele)
        
        #Se tiene la lista "lista_value_type_constraints" con las value type constraint de la propiedad estudiada.
        if (lista_clases_para_una_propiedad != []) & (lista_value_type_constraints != []):
            #Se guardan en la lista "lista_clases_a_representar" aquellos identificadores que esten en ambas listas.
            lista_clases_a_representar = []
            for ele_VTC in lista_value_type_constraints:
                for ele_CPUP in lista_clases_para_una_propiedad:
                    if ele_VTC == ele_CPUP:
                        lista_clases_a_representar.append(ele_VTC)
            
            #Se tiene la lista "lista_clases_a_representar" con las clases que hay que representar para la propiedad estudiada.
            
            #Se eliminan los duplicados
            if lista_clases_a_representar != []:             
                result = []
                for ele in lista_clases_a_representar:
                    if ele not in result:
                        result.append(ele)
                        
                #Se añade a la lista final el valor de la propiedad
                lista_resultado_propiedades_clases.append(prop)
                
                result2 = result
                result3 = result
                #Se eliminan las clases que sean subclases de otra clase de la misma lista
                if len(result) != 1:
                    for ele in result:
                        #Se obtienen las subclases de la clase ele
                        sparql3 = SPARQLWrapper("https://query.wikidata.org/sparql")
                        sparql3.setQuery("""SELECT ?s ?desc
                        WHERE{
                        ?s wdt:P279+ wd:"""+ele+""" .
                        OPTIONAL {
                        ?s rdfs:label ?desc filter (lang(?desc) = "es").}
                        }
                        """)
                        sparql3.setReturnFormat(JSON)
                        results = sparql3.query().convert()

                        #En la lista "lista_value_type_constraints_completa" se almacenan todas las value type constraint de 
                        #la propiedad estudiada.
                        lista_subclases_de_ele_completa = []
                        for result in results["results"]["bindings"]:
                            lista_subclases_de_ele_completa.append(result["s"]["value"])
                           
                        #Al tener estos valores en su inicio la URL "https:..", esto se elimina dejando solo el identificador 
                        #de la clase.
                        lista_subclases_de_ele = []
                        for ele2 in lista_subclases_de_ele_completa:
                            ele2 = ele2[31:]
                            lista_subclases_de_ele.append(ele2)
                          
                        #Se comparan todos los elementos de la lista "result2" con los de la lista "lista_subclase_de_ele" y 
                        #se eliman de la lista final si estos son iguales.
                        for ele4 in result2:
                            for ele3 in lista_subclases_de_ele:
                                if ele4 == ele3:
                                    result3.remove(ele4)
                
                #Se añaden las listas con las clases a representar en la lista final.
                lista_resultado_propiedades_clases.append(result3)
                
#Se imprime la lista resultado final con las propiedades a representar y sus respectivas clases
print(lista_resultado_propiedades_clases)

['P276', ['Q82794'], 'P495', ['Q6256', 'Q3024240'], 'P641', ['Q31629'], 'P910', ['Q4167836'], 'P6104', ['Q16695773'], 'P17', ['Q6256', 'Q43702', 'Q3024240'], 'P452', ['Q268592', 'Q8187769'], 'P488', ['Q5'], 'P2596', ['Q41710'], 'P170', ['Q5']]


In [447]:
a=ET.Element('mxfile')
a.set('type','device')
a.set('etag','d2m_BCRs5NTD1uJDUOON')
a.set('version','13.9.1')
a.set('agent','5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36')
a.set('modified','2020-11-05T11:56:57.085Z')
a.set('host','app.diagrams.net')

b = ET.SubElement(a, 'diagram')
b.set('id','E2t0QyFFZp_JITEdpmB3')
iteracion = 0

c = ET.SubElement(b, 'mxGraphModel')
c.set('shadow','0')
c.set('math','0')
c.set('pageHeight','1169')
c.set('pageWidth','827')
c.set('pageScale','1')
c.set('page','1')
c.set('fold','1')
c.set('arrows','1')
c.set('connect','1')
c.set('tooltips','1')
c.set('guides','1')
c.set('gridSize','10')
c.set('grid','1')
c.set('dy','18481')
c.set('dx','21749')

d = ET.SubElement(c, 'root')

e = ET.SubElement(d, 'mxCell')
e.set('id','0')
f = ET.SubElement(d, 'mxCell')
f.set('id','1')
f.set('parent','0')

#Caja con la clase inicial
g = ET.SubElement(d, 'mxCell')
g.set('id','2')
g.set('parent','1')
g.set('style','rounded=0;whiteSpace=wrap;html=1;fillColor=#fff2cc;strokeColor=#d6b656')
g.set('vertex','1')
g.set('value',clase)
g1 = ET.SubElement(g, 'mxGeometry')
g1.set('as','geometry')
g1.set('y','-16200')
g1.set('x','-18500')
g1.set('height','27')
g1.set('width','212')

identificador = 3
for i in range(0,len(lista_resultado_propiedades_clases)):
    if iteracion % 2 == 0:
        
        #Estamos tratando una propiedad
        #Flecha con la propiedad
        k = ET.SubElement(d, 'mxCell')
        k.set('id',str(identificador))
        k.set('parent','1')
        k.set('target',str(identificador+1))
        k.set('source','2')
        k.set('edge','1')
        k.set('style','edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;exitX=1;exitY=0.5;exitDx=0;exitDy=0;dashed=1;endArrow=classic;endFill=1;')
        k.set('value',lista_resultado_propiedades_clases[i])
        k1 = ET.SubElement(k, 'mxGeometry')
        k1.set('as','geometry')
        k1.set('relative','1')
        
    else:
        #Estamos tratando una lista de clases
        num_elem = 1
        for j in lista_resultado_propiedades_clases[i]:
            if num_elem > 1:
                identificador = identificador +1
            #Caja con clase
            z = ET.SubElement(d, 'mxCell')
            z.set('id',str(identificador))
            z.set('parent','1')
            z.set('style','rounded=0;whiteSpace=wrap;html=1')
            z.set('vertex','1')
            z.set('value',j)
            z1 = ET.SubElement(z, 'mxGeometry')
            z1.set('as','geometry')
            z1.set('y','-16800')
            z1.set('x','-19450')
            z1.set('height','27')
            z1.set('width','212')
            num_elem = num_elem + 1
        
    iteracion = iteracion + 1
    identificador = identificador + 1
                       
ET.dump(a)
tree = ET.ElementTree(a)
tree.write("output_final.xml", encoding='utf-8', xml_declaration=True)                       

<mxfile agent="5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36" etag="d2m_BCRs5NTD1uJDUOON" host="app.diagrams.net" modified="2020-11-05T11:56:57.085Z" type="device" version="13.9.1"><diagram id="E2t0QyFFZp_JITEdpmB3"><mxGraphModel arrows="1" connect="1" dx="21749" dy="18481" fold="1" grid="1" gridSize="10" guides="1" math="0" page="1" pageHeight="1169" pageScale="1" pageWidth="827" shadow="0" tooltips="1"><root><mxCell id="0" /><mxCell id="1" parent="0" /><mxCell id="2" parent="1" style="rounded=0;whiteSpace=wrap;html=1;fillColor=#fff2cc;strokeColor=#d6b656" value="Q349" vertex="1"><mxGeometry as="geometry" height="27" width="212" x="-18500" y="-16200" /></mxCell><mxCell edge="1" id="3" parent="1" source="2" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;exitX=1;exitY=0.5;exitDx=0;exitDy=0;dashed=1;endArrow=classic;endFill=1;" target="4" value="P276"><mxGeometry as="geometry" relative="1"

Q82794
